## Baixa Instruções Normativas de Regulação

In [ ]:
# ======================================================================
# ETAPA 1: Instalar bibliotecas (caso precise no ambiente do Jupyter)
# Se você já tiver alguma instalada, pode pular
# ======================================================================

#!pip install requests
#!pip install beautifulsoup4
#!pip install xhtml2pdf


In [ ]:
# ======================================================================
# ETAPA 2: Importar bibliotecas
# ======================================================================

import os
import re
import time
import requests
from bs4 import BeautifulSoup, Tag
from xhtml2pdf import pisa
from urllib.parse import urljoin
import win32com.client

In [ ]:
# --------------------------------------------------
# CONFIGURAÇÕES E NOMES DE ARQUIVOS
# --------------------------------------------------
URL_BASE = "https://www.gov.br/ancine/pt-br/acesso-a-informacao/legislacao/instrucoes-normativas"
PASTA_SAIDA = "INs_Ancine_PDFs"
# Lista de números das INs a serem baixadas
LISTA_INS = [170, 169, 168, 165, 164, 163, 162, 157, 156, 155, 153, 151, 148, 147, 146, 139, 
             134, 129, 128, 127, 123, 122, 121, 120, 119, 118, 115, 112, 111, 109, 107, 106, 
             105, 104, 102, 101, 100, 98, 97, 96, 95, 94, 92, 91, 84, 79, 63, 60, 57]

# Cria a pasta de saída se não existir
if not os.path.exists(PASTA_SAIDA):
    os.makedirs(PASTA_SAIDA)


In [ ]:
# --------------------------------------------------
# FUNÇÃO AUXILIAR: CONVERTER HTML PARA PDF
# --------------------------------------------------
def converter_html_para_pdf(html_string, output_path):
    """
    Gera um PDF a partir de uma string HTML usando xhtml2pdf (pisa).
    Retorna True em caso de sucesso, False caso contrário.
    """
    with open(output_path, "wb") as pdf_file:
        pisa_status = pisa.CreatePDF(
            html_string,
            dest=pdf_file,
            encoding="utf-8"
        )
    return pisa_status.err == 0


In [ ]:
# --------------------------------------------------
# FUNÇÃO: VERIFICAR SE UMA URL EXISTE
# --------------------------------------------------
def verificar_url(url):
    """
    Verifica se uma URL existe fazendo uma requisição HEAD
    Retorna True se a URL existir, False caso contrário
    """
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.head(url, headers=headers, timeout=10)
        return resp.status_code == 200
    except:
        return False

# --------------------------------------------------
# FUNÇÃO: OBTER LINKS DAS INSTRUÇÕES NORMATIVAS
# --------------------------------------------------
def obter_links_ins():
    """
    Acessa a página principal de INs e retorna um dicionário com número da IN e respectivo link
    """
    print(f"Acessando página principal: {URL_BASE}")
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        resp = requests.get(URL_BASE, headers=headers, timeout=30)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"ERRO: Falha ao acessar página principal. Detalhes: {e}")
        return {}
    
    links_ins = {}
    soup = BeautifulSoup(resp.text, "lxml")
    
    # Procurar todos os links que podem conter Instruções Normativas
    for link in soup.find_all("a"):
        href = link.get("href", "")
        text = link.get_text(strip=True)
        
        # Tenta encontrar o número da IN no texto ou no href
        num_match = re.search(r'(?:instrucao|instrução)(?:-|\s)normativa(?:-|\s)(?:n[oº]?\s*)?(\d+)', 
                              href.lower() + " " + text.lower())
        
        if num_match:
            num_in = int(num_match.group(1))
            if num_in in LISTA_INS:
                # Converte URLs relativas para absolutas
                url_absoluta = urljoin(URL_BASE, href)
                links_ins[num_in] = url_absoluta
                print(f"    Encontrado link para IN {num_in}")
    
    # Para as INs que não foram encontradas, tentar diferentes padrões de URL
    for num_in in LISTA_INS:
        if num_in not in links_ins:
            # Tentar diferentes formatos de URL, começando pelo mais provável
            formatos = [
                f"{URL_BASE}/instrucao-normativa-no-{num_in}",         # Com "no"
                f"{URL_BASE}/instrucao-normativa-{num_in}",            # Sem "no"
                f"{URL_BASE}/instrucao-normativa-n-{num_in}",          # Com "n"
                f"{URL_BASE}/instrucao-normativa-{num_in:03d}",        # Com zeros à esquerda
                f"{URL_BASE}/instrucao-normativa-no-{num_in:03d}"      # Com "no" e zeros à esquerda
            ]
            
            # Testar cada formato e usar o primeiro que funcionar
            for url in formatos:
                print(f"    Testando URL para IN {num_in}: {url}")
                if verificar_url(url):
                    links_ins[num_in] = url
                    print(f"    ✓ URL válida encontrada para IN {num_in}: {url}")
                    break
            
            # Se nenhum formato funcionou, usar o primeiro como padrão
            if num_in not in links_ins:
                links_ins[num_in] = formatos[0]
                print(f"    ⚠ Nenhuma URL válida encontrada para IN {num_in}. Usando URL padrão.")
    
    return links_ins


In [ ]:
# --------------------------------------------------
# FUNÇÃO PRINCIPAL: BAIXAR E PROCESSAR UMA IN
# --------------------------------------------------
def baixar_in(url, numero_in):
    """
    Baixa e processa uma Instrução Normativa específica
    """
    nome_pdf = f"IN_{numero_in}_consolidada.pdf"
    caminho_pdf = os.path.join(PASTA_SAIDA, nome_pdf)
    
    print(f"\n[IN {numero_in}] Iniciando download:")
    print(f"    URL = {url}")
    
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        resp = requests.get(url, headers=headers, timeout=30)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"ERRO: Falha ao baixar HTML da IN {numero_in}. Detalhes: {e}")
        
        # Se falhar com a URL atual, tentar URL alternativa
        if "instrucao-normativa-no-" in url:
            url_alternativa = url.replace("instrucao-normativa-no-", "instrucao-normativa-")
            print(f"    Tentando URL alternativa: {url_alternativa}")
            try:
                resp = requests.get(url_alternativa, headers=headers, timeout=30)
                resp.raise_for_status()
                url = url_alternativa  # Se funcionou, atualiza a URL
            except requests.RequestException:
                print(f"    Também falhou com URL alternativa.")
                return False
        elif "instrucao-normativa-" in url and "no-" not in url:
            url_alternativa = url.replace("instrucao-normativa-", "instrucao-normativa-no-")
            print(f"    Tentando URL alternativa: {url_alternativa}")
            try:
                resp = requests.get(url_alternativa, headers=headers, timeout=30)
                resp.raise_for_status()
                url = url_alternativa  # Se funcionou, atualiza a URL
            except requests.RequestException:
                print(f"    Também falhou com URL alternativa.")
                return False
        else:
            return False
    
    html = resp.text
    print(f"[IN {numero_in}] Download concluído. Tamanho aproximado do HTML:", len(html), "caracteres.")

    print(f"[IN {numero_in}] Iniciando parse do HTML com BeautifulSoup...")
    soup = BeautifulSoup(html, "lxml")
    print(f"    Parse concluído com sucesso.")

    # --------------------------------------------------
    # A) LOCALIZAR CONTEÚDO PRINCIPAL
    # --------------------------------------------------
    print(f"[IN {numero_in}] Buscando container principal...")
    content_div = soup.find("div", id="content-core")
    if not content_div:
        content_div = soup.find("div", attrs={"property": "rnews:articleBody"})
    if not content_div:
        content_div = soup.find("div", id="content")
    if not content_div:
        # Tenta encontrar pelo título ou classe comum
        for classe in ["documentContent", "entry-content", "document-content", "main-content"]:
            content_div = soup.find("div", class_=classe)
            if content_div:
                break
                
        # Se ainda não encontrou, tenta pelo título
        if not content_div:
            for heading in soup.find_all(["h1", "h2", "h3"]):
                text = heading.get_text(strip=True).lower()
                if re.search(r'instruç[ãa]o\s+normativa', text):
                    content_div = heading.parent
                    break
    
    if not content_div:
        print(f"ERRO: Não foi encontrado conteúdo para IN {numero_in}.")
        return False
    
    print(f"    Container principal encontrado!")

    # --------------------------------------------------
    # B) REMOVER TEXTO TACHADO
    # --------------------------------------------------
    print(f"[IN {numero_in}] Removendo texto tachado...")
    # 1. Apagar tags específicas <s>, <del>, <strike>
    tcount = 0
    for tag in content_div.find_all(["s", "del", "strike"]):
        tag.decompose()
        tcount += 1

    # 2. Apagar elementos que usem style=line-through
    scount = 0
    for elm in content_div.find_all():
        # Verificar se elm é None ou não é uma Tag antes de acessar attrs
        if elm is None or not hasattr(elm, 'attrs'):
            continue
            
        # Verificar se attrs existe e se é um dicionário antes de usar .get()
        if not hasattr(elm, 'attrs') or not isinstance(elm.attrs, dict):
            continue
            
        # Agora é seguro usar .get()
        style_attr = elm.attrs.get("style", "")
        if isinstance(style_attr, str) and "line-through" in style_attr.lower():
            elm.decompose()
            scount += 1

    print(f"    -> {tcount} tags <s>/<del>/<strike> removidas.")
    print(f"    -> {scount} elementos com style='line-through' removidos.")

    # --------------------------------------------------
    # C) REMOVER PARÊNTESES QUE CONTENHAM 'Revogado'
    # --------------------------------------------------
    print(f"[IN {numero_in}] Removendo parênteses com 'Revogado'...")
    pcount = 0
    paragraphs = content_div.find_all("p")
    for p in paragraphs:
        original_text = p.get_text()
        # Remover apenas se contiver 'Revogado' dentro dos parênteses:
        novo_texto = re.sub(r"\(\s*[Rr]evogado.*?\)", "", original_text, flags=re.DOTALL)
        if novo_texto != original_text:
            pcount += 1
        # Eliminar espaços duplicados
        novo_texto = re.sub(r"\s+", " ", novo_texto).strip()

        p.clear()
        p.append(novo_texto)

    print(f"    -> {pcount} parágrafos modificados por '(Revogado...)'.")

    # --------------------------------------------------
    # D) OBTER TÍTULO PARA USAR NO PDF
    # --------------------------------------------------
    print(f"[IN {numero_in}] Buscando título da IN...")
    titulo_encontrado = None
    for heading in soup.find_all(["h1", "h2"]):
        text_heading = heading.get_text(strip=True) if heading else ""
        if re.search(r"instrução normativa", text_heading, re.IGNORECASE):
            titulo_encontrado = text_heading
            break
    if not titulo_encontrado:
        titulo_encontrado = f"Instrução Normativa ANCINE nº {numero_in}"
        print(f"    -> Título padrão adotado (não encontrado no HTML).")
    else:
        print(f"    -> Título encontrado: {titulo_encontrado}")

    # --------------------------------------------------
    # E) MONTAR HTML FINAL E GERAR PDF
    # --------------------------------------------------
    print(f"[IN {numero_in}] Montando HTML final para conversão em PDF...")

    html_final = f"""<!DOCTYPE html>
<html lang="pt-br">
<head>
  <meta charset="UTF-8"/>
  <title>{titulo_encontrado}</title>
  <style>
    body {{
      font-family: Arial, sans-serif;
      margin: 40px;
      line-height: 1.4;
    }}
    p {{
      text-align: justify;
      margin-bottom: 10px;
    }}
    h1, h2 {{
      text-align: center;
      margin-bottom: 20px;
    }}
  </style>
</head>
<body>
  <h2>{titulo_encontrado}</h2>
  {content_div.prettify()}
</body>
</html>
"""

    print(f"[IN {numero_in}] Convertendo HTML para PDF...\n    Saída em: {caminho_pdf}")
    ok = converter_html_para_pdf(html_final, caminho_pdf)
    if ok:
        print(f"    -> PDF gerado com sucesso: {caminho_pdf}")
        return True
    else:
        print(f"    ERRO: Falha ao gerar o PDF para IN {numero_in}.")
        return False


In [ ]:
# --------------------------------------------------
# FUNÇÃO PARA BAIXAR TODAS AS INS DA LISTA
# --------------------------------------------------
def baixar_todas_ins():
    """
    Baixa todas as INs da lista
    """
    print("*** INICIANDO DOWNLOAD DE MÚLTIPLAS INSTRUÇÕES NORMATIVAS ***")
    links = obter_links_ins()
    
    sucessos = 0
    falhas = 0
    
    for numero_in in LISTA_INS:
        if numero_in in links:
            url = links[numero_in]
            print(f"\n=== PROCESSANDO IN {numero_in} ===")
            try:
                resultado = baixar_in(url, numero_in)
                if resultado:
                    sucessos += 1
                else:
                    falhas += 1
            except Exception as e:
                print(f"ERRO INESPERADO ao processar IN {numero_in}: {str(e)}")
                falhas += 1
                
            # Pequena pausa entre downloads para não sobrecarregar o servidor
            time.sleep(2)
        else:
            print(f"\nAVISO: Link para IN {numero_in} não encontrado.")
            falhas += 1
    
    print("\n*** RESUMO FINAL ***")
    print(f"Total de INs processadas: {sucessos + falhas}")
    print(f"INs baixadas com sucesso: {sucessos}")
    print(f"INs com falha: {falhas}")
    print(f"PDFs salvos no diretório: {PASTA_SAIDA}")



In [ ]:
# --------------------------------------------------
# EXECUÇÃO
# --------------------------------------------------
if __name__ == "__main__":
    baixar_todas_ins()

## Baixa leis

In [ ]:
# Baixa em pdf lei de criação da Ancine (MP2281-1) e a Lei do SeAC (lei da tv paga)


# === Processamento da Medida Provisória ===

# Exemplo de URL do arquivo .doc
url_doc_mp = "https://www2.camara.leg.br/legin/fed/medpro/2001/medidaprovisoria-2228-1-6-setembro-2001-404188-normaatualizada-pe.doc"

# Definindo os nomes dos arquivos
nome_arquivo_doc_mp = "medida_provisoria_2228-1.doc"
nome_arquivo_pdf_mp = "medidaprovisoria-2228-1.pdf"

# Monta os caminhos completos para os arquivos dentro da subpasta (a pasta utilizada é exibida pelo seu nome)
caminho_doc_mp = os.path.join(PASTA_SAIDA, nome_arquivo_doc_mp)
caminho_pdf_mp = os.path.join(PASTA_SAIDA, nome_arquivo_pdf_mp)

# Download do arquivo .doc
response = requests.get(url_doc_mp)
with open(caminho_doc_mp, "wb") as file:
    file.write(response.content)
print(f"Arquivo '{nome_arquivo_doc_mp}' baixado com sucesso na subpasta {PASTA_SAIDA}.")

# Converter .doc para .pdf utilizando automação do Microsoft Word (funciona apenas no Windows com Word instalado)
word = win32com.client.Dispatch("Word.Application")
word.Visible = False  # Executa o Word em modo invisível

# Abre o documento .doc (é necessário passar o caminho absoluto)
doc = word.Documents.Open(os.path.abspath(caminho_doc_mp))

# Salva o documento como PDF (FileFormat=17 corresponde a PDF)
doc.SaveAs(os.path.abspath(caminho_pdf_mp), FileFormat=17)
doc.Close()
word.Quit()

print(f"Arquivo convertido com sucesso para '{nome_arquivo_pdf_mp}' na subpasta {PASTA_SAIDA}.")

# Após a conversão, remove o arquivo .doc
try:
    os.remove(caminho_doc_mp)
    print(f"Arquivo '{nome_arquivo_doc_mp}' removido com sucesso da subpasta {PASTA_SAIDA}.")
except Exception as e:
    print(f"Erro ao remover o arquivo '{nome_arquivo_doc_mp}':", e)


# === Processamento da Lei 12.485 ===

# Exemplo de URL do arquivo .doc da lei
url_doc_lei = "https://www2.camara.leg.br/legin/fed/lei/2011/lei-12485-12-setembro-2011-611410-normaatualizada-pl.doc"

# Definindo os nomes dos arquivos
nome_arquivo_doc_lei = "lei_12485.doc"
nome_arquivo_pdf_lei = "lei-12485-12-setembro-2011.pdf"

# Monta os caminhos completos para os arquivos dentro da subpasta (exibindo o nome da pasta)
caminho_doc_lei = os.path.join(PASTA_SAIDA, nome_arquivo_doc_lei)
caminho_pdf_lei = os.path.join(PASTA_SAIDA, nome_arquivo_pdf_lei)

# Download do arquivo .doc
response = requests.get(url_doc_lei)
with open(caminho_doc_lei, "wb") as file:
    file.write(response.content)
print(f"Arquivo '{nome_arquivo_doc_lei}' baixado com sucesso na subpasta {PASTA_SAIDA}.")

# Converter .doc para .pdf utilizando automação do Microsoft Word
word = win32com.client.Dispatch("Word.Application")
word.Visible = False  # Executa o Word em modo invisível

doc = word.Documents.Open(os.path.abspath(caminho_doc_lei))
doc.SaveAs(os.path.abspath(caminho_pdf_lei), FileFormat=17)
doc.Close()
word.Quit()

print(f"Arquivo convertido com sucesso para '{nome_arquivo_pdf_lei}' na subpasta {PASTA_SAIDA}.")

# Aguarda um instante para garantir que o sistema libere o arquivo, se necessário
time.sleep(1)

# Após a conversão, remove o arquivo .doc
try:
    os.remove(caminho_doc_lei)
    print(f"Arquivo '{nome_arquivo_doc_lei}' removido com sucesso da subpasta {PASTA_SAIDA}.")
except Exception as e:
    print(f"Erro ao remover o arquivo '{nome_arquivo_doc_lei}':", e)
